In [39]:
# checking connection via iio library and looking at internal devices
# an iio context has internal attributes as well as 1 or more devices, then
# each iio device has internal attributes as well as 1 or more channels, and
# channels are for streaming data, input or output with attributes for control 

In [40]:
import iio
pluto = iio.Context('ip:pluto.local')            # connect to pluto hardware

In [41]:
# there are 5 devices in the pluto iio context each with various channels
for dev in pluto.devices:
    print('{:24s}{:d}'.format(dev.name, len(dev.channels)))
    
# there are 3 relevant RF devices:
# ad9361-phy       where the internals control the analogue RF params of rx and rx 
#                  e.g. filter BWs, gain, agc, LO 
# cf-ad9361-lpc    where the 2 channels are the IQ data from the ADC
# cf-ad9361-dds... where 2 channels are the DAC IQ and the other 4 are IQ, F1/F2 of the 

adm1177                 2
ad9361-phy              9
xadc                    10
cf-ad9361-dds-core-lpc  6
cf-ad9361-lpc           2


In [42]:
# picking these out for futher examination
phy = pluto.find_device('ad9361-phy')
rx = pluto.find_device('cf-ad9361-lpc')
tx = pluto.find_device('cf-ad9361-dds-core-lpc')

In [43]:
# in the phy, these are the most interesting
rx_control = phy.find_channel('voltage0', is_output=False)
tx_control = phy.find_channel('voltage0', is_output=True)

In [44]:
# the attrs dict shows what can be controlled
print('rx control\n', rx_control.attrs.keys())
print('tx control\n', tx_control.attrs.keys())

rx control
 dict_keys(['bb_dc_offset_tracking_en', 'filter_fir_en', 'gain_control_mode', 'gain_control_mode_available', 'hardwaregain', 'hardwaregain_available', 'quadrature_tracking_en', 'rf_bandwidth', 'rf_bandwidth_available', 'rf_dc_offset_tracking_en', 'rf_port_select', 'rf_port_select_available', 'rssi', 'sampling_frequency', 'sampling_frequency_available'])
tx control
 dict_keys(['filter_fir_en', 'hardwaregain', 'hardwaregain_available', 'rf_bandwidth', 'rf_bandwidth_available', 'rf_port_select', 'rf_port_select_available', 'rssi', 'sampling_frequency', 'sampling_frequency_available'])


In [45]:
# for the rx, the channels are IQ data paths with attrs for dc offset (bias) and phase calibration
print('rx_adc\n', rx.channels[0].attrs.keys())

rx_adc
 dict_keys(['calibbias', 'calibphase', 'calibscale', 'samples_pps', 'sampling_frequency', 'sampling_frequency_available'])


In [46]:
# for the tx, 2 of the channels are IQ data paths for the dac
tx_dac = tx.find_channel('voltage0', True)
print('tx_dac\n', tx_dac.attrs.keys())
# the Q channel is 'voltage1'

tx_dac
 dict_keys(['calibphase', 'calibscale', 'sampling_frequency', 'sampling_frequency_available'])


In [47]:
# the others control internally generated IQ data for F1 and/or F2 DDS signals 
tx_f1 = tx.find_channel('altvoltage0', True)
print('dds_F1\n', tx_f1.attrs.keys())
tx_f2 = tx.find_channel('altvoltage1', True)
print('dds_F1\n', tx_f2.attrs.keys())
# the corresponding Q channels are altvoltage2 and altvoltage3

dds_F1
 dict_keys(['frequency', 'phase', 'raw', 'sampling_frequency', 'scale'])
dds_F1
 dict_keys(['frequency', 'phase', 'raw', 'sampling_frequency', 'scale'])
